# ECS529U Algorithms and Data Structures
# Lab sheet 9

This lab gets you to work with hash tables.

**Marks (max 5):**  Question 1: 1.5 | Questions 3-5: 0.5 each | Questions 2,6: 1 each

## Question 1

Consider hash tables with underlying array of size 7, and (initial) hash function the remainder of dividing by 7:

    def hash(d): return i%7

a) Draw the hash table we obtain if we start from the empty one and add consecutively the numbers: 

    412, 21, 420, 3, 56, 99, 1528, 21, 462, 409

without resizing up the underlying array.

```
0 [462, 21, 56, 420, 21]
1 [99]
2 [1528]
3 [409, 3]
4 []
5 []
6 [412]
```

b) Draw the hash table we obtain if we start from the empty one and add consecutively the same numbers as above, but using resizing of the underlying array whenever the load factor exceeds 0.75. 

The hash function should be updated accordingly after each resize operation.

```
0 [462, 56, 420]
1 [99]
2 [1528]
3 [409, 3]
4 []
5 []
6 [412]
7 [21, 21]
8 []
9 []
10 []
11 []
12 []
13 []
```

In [4]:
class Node:
    def __init__(self, d, n):
        self.data = d
        self.next = n

# A version of LinkedList useful for HashTables/HashMaps
class LinkedList:
    def __init__(self):
        self.head = None
        self.length = 0

    def __str__(self):
        st = ""
        ptr = self.head
        while ptr != None:
            st += " -> " + str(ptr.data)
            ptr = ptr.next
        return st

    def toSeqString(self):
        st = ""
        ptr = self.head
        while ptr != None:
            st += str(ptr.data)+", "
            ptr = ptr.next
        if st == "": return st
        return st[:len(st)-2]

    def search(self, d):
        i = 0
        ptr = self.head
        while ptr != None:
            if ptr.data == d:
                return i
            ptr = ptr.next
            i += 1
        return -1
        
    def get(self, i):
        ptr = self.head
        while ptr != None and i>0:
            ptr = ptr.next
            i -= 1
        return ptr.data
    
    def insert(self, i, d):
        if self.head == None or i == 0:
            self.head = Node(d,self.head)
        else:
            ptr = self.head
            while i>1 and ptr.next != None:
                ptr = ptr.next
                i -= 1
            ptr.next = Node(d,ptr.next)
        self.length += 1

    def append(self, d):
        if self.head == None:      
            self.head = Node(d,None) 
        else:
            ptr = self.head
            while ptr.next != None:
                ptr = ptr.next
            ptr.next = Node(d,None)
        self.length += 1

    def remove(self, i): # removes i-th element and returns it
        if self.head == None:
            return None
        if i == 0:
            val = self.head.data
            self.head = self.head.next
            self.length -= 1
            return val
        else:
            ptr = self.head
            while i>1 and ptr.next != None:
                ptr = ptr.next
                i -= 1
            if i == 1:
                val = ptr.next.data
                ptr.next = ptr.next.next
                self.length -= 1
                return val
            return None
    
    def removeVal(self, d):
        if self.head == None:
            return
        if self.head.data == d:
            self.head = self.head.next
            self.length -= 1
        else:
            ptr = self.head
            while ptr.next != None:
                if ptr.next.data == d:
                    ptr.next = ptr.next.next
                    self.length -= 1
                    break
                ptr = ptr.next

## Question 2

Add in `HashTable` (the code of which you can find in `lecture9.ipynb`) functions:

    a) def min(self)

that returns the smallest of all elements in the hashtable (represented by `self`).

    b) def toArray(self)

that returns an array containing all elements in the hashtable (represented by `self`),
in no particular order. For example, if `h` is a hashtable constructed by:

    h = HashTable(); h.addAll([5,10,0,42,42])
    
then `h.toArray()` should return an array that contains the same elements as the array `[5,10,0,42,42]`.

## Question 3

Add in `HashTable` a function:

    def __str__(self)

that returns a string representing the hashtable in the following format. 

- The string should contain a sequence of lines, one for each entry of the underlying array, separated by newlines ('\n'). 
- Each of these lines should start with the index of the underlying array, included in square brackets, followed by a string representation of the linked list in that index.

For example, if `h` is a hashtable with an underlying array of size 7 constructed by:

    h = HashTable(7); h.addAll([42,2049,12,5,18])
    
then, `str(h)` should return the string:

    [0] -> 42\n[1]\n[2]\n[3]\n[4] -> 18\n[5] -> 5 -> 12 -> 2049\n[6]\n'

And, therefore, `print(h)` would print:

    [0] -> 42
    [1]
    [2]
    [3]
    [4] -> 18
    [5] -> 5 -> 12 -> 2049
    [6]
    
__Hint:__ You might find useful to replace the `__str__` function of `LinkedList` with one that is more suitable for this task.

## Question 4

Re-implement the `HashTable` function

    def hash(self, d)

so that, for each input integer `d`, it adds together all the digits of `d` and takes the remainder 
of dividing the resulting number by the length of the internal array. For example, if `h` is built 
as follows:

    h = HashTable(7); h.addAll([42,2049,12,5,18])
    
then `print(h)` should print (using your implementation of `str` from Question 3):

    [0]
    [1] -> 2049
    [2] -> 18
    [3] -> 12
    [4]
    [5] -> 5
    [6] -> 42

    
## Question 5
Re-implement the `HashTable` function

    def hash(self, d)

so that it works for inputs d of arbitrary type (e.g. Strings, LinkedLists, anything). For that, 
you can use Python’s built-in `hash` function. For example, if `h` is built as follows:

    h = HashTable(7); h.addAll(["Algorithms","And","Data Structures",42,16])

then `print(h)` should print a variant of (using your implementation of `str`):

    [0] -> 42 -> Data Structures
    [1]
    [2] -> 16 -> Algorithms
    [3]
    [4] -> And
    [5]
    [6]

In [2]:
import sys

class HashTable:
    def __init__(self, d=10):    # a default array length of 10
        self.inArray = [LinkedList() for i in range(d)] 
        self.size = 0
        self.threshold = 0.75    # a default threshold of 0.75

    def __str__(self):
        s = ''
        for i, l in enumerate(self.inArray):
            s += f'[{i}]'
            ptr = l.head
            while ptr:
                s += f' -> {ptr.data}'
                ptr = ptr.next
            s += '\n'
        return s

    def hash(self, d):
        return d % len(self.inArray)

           
    def add(self, d):
        i = self.hash(d)
        self.inArray[i].insert(0,d)
        self.size += 1
        # condition that must be respected: self.size <= self.threshold * len(self.inArray)
        if self.size > self.threshold*len(self.inArray): self._resizeUp()
        
    def search(self, d):
        i = self.hash(d)
        if self.inArray[i].search(d) == -1: return False
        return True

    def remove(self, d):
        i = self.hash(d)
        oldLength = self.inArray[i].length
        self.inArray[i].removeVal(d)
        if self.inArray[i].length < oldLength:
            self.size -= 1

    def _resizeUp(self):
        oldArray = self.inArray
        self.inArray = [LinkedList() for _ in range(2*len(oldArray))]
        for l in oldArray:
            while l.length > 0:
                d = l.remove(0)
                self.inArray[self.hash(d)].insert(0,d)            
            
    # these are to make our life easier in adding/removing many elements
    def addAll(self, A):
        for i in range(len(A)):
            self.add(A[i])

    def removeAll(self, A):
        for i in range(len(A)):
            self.remove(A[i])

    def min(self):
        r = sys.maxsize
        for l in self.inArray:
            ptr = l.head
            while ptr:
                if ptr.data < r:
                    r = ptr.data
                ptr = ptr.next
        return r

    def toArray(self):
        count = 0
        for l in self.inArray:
            count += l.length
        a = [0 for _ in range(count)]
        count = 0
        for l in self.inArray:
            if l.head:
                ptr = l.head
                while ptr:
                    a[count] = ptr.data
                    count += 1
                    ptr = ptr.next
        return a

class HashTableDigit(HashTable):

    def __init__(self, d=10):
        super().__init__(d)

    def hash(self, d):
        s = 0
        while d:
            s += d % 10
            d //= 10
        return s % len(self.inArray)

class HashTableArbit(HashTable):

    def __init__(self, d=10):
        super().__init__(d)

    def hash(self, d):
        return hash(d) % len(self.inArray)
        
h = HashTable()
h.addAll([3, 0, 6, 2, 4, 7, 0])
print('Min:', h.min())
print('Array:', h.toArray())
print(h)
h = HashTableDigit(7)
h.addAll([42,2049,12,5,18])
print('Q4:', h)
h = HashTableArbit(7)
h.addAll(["Algorithms","And","Data Structures",42,16])
print('Q5:', h)

Min: 0
Array: [0, 0, 2, 3, 4, 6, 7]
[0] -> 0 -> 0
[1]
[2] -> 2
[3] -> 3
[4] -> 4
[5]
[6] -> 6
[7] -> 7
[8]
[9]

Q4: [0]
[1] -> 2049
[2] -> 18
[3] -> 12
[4]
[5] -> 5
[6] -> 42

Q5: [0] -> 42 -> Data Structures
[1]
[2] -> 16
[3]
[4]
[5] -> And -> Algorithms
[6]



## Hash Maps

In the last question we will implement hash maps, which are hash tables that store objects 
that are pairs of the form: (key,value). Each key in the hash map has at most one 
corresponding value. Hashing of such pairs (in order to search for them, remove them, etc.) is done with respect
to their key. That is, to search if there is a pair with key 42, we need to hash 42 to retrieve 
an index, and then search the linked list in the corresponding entry of the internal array for 
a pair with key 42. 

To store pairs, we will use the following class, where note that we take two pairs to be 
equal just if their keys are equal. 

    class KVPair:
        def __init__(self, k, v):
            self.key = k
            self.val = v
        def __eq__(self, other):
            return self.key == other.key

## Question 6

Adapt the implementation of `HashTable` into a class `HashMap` with functions:

- `def search(self,k)` : searches for a pair with key `k` in the hash map and returns 
`True` if it finds one, otherwise `False`.
- `def remove(self,k)` : searches for a pair with key `k` in the hash map and removes 
it if it finds its, otherwise it leaves the hash map unchanged.
- `def get(self,k)` : returns the value `v` with which the key `k` is paired in the hash 
map (i.e. such that `(k,v)` is in the hash map). If there is no such pair, returns `None`.
- `def put(self,k,v)` : adds the pair `(k,v)` in the hash map. If there is already a pair 
with key `k` in the map, then its value is updated to `v`.

Your hash maps should work with keys and values of arbitrary type (so, you would need to  use a hash function like the one in Question 5). 

To help you with the implementation, we have made a start below and implemented the constructor and the `search` function. In the same file, we have 
included an implementation of `LinkedList` with a couple of additional functions that can be
of help (`toSeqString`, for printing a linked list as a comma-separated sequence, and `get`).

For example, executing the following code:

    h = HashMap()
    h.put(10,12); print(h, h.size, h.search(10), h.search(42))
    h.put(10,13); h.put(42,12); print(h, h.size, h.search(10), h.search(42))
    h.put("Algorithms","Are Great!"); print(h)

we should get the printout:

    {(10, 12)} 1 True False
    {(10, 13),(42, 12)} 2 True True
    {(10, 13),(42, 12),(Algorithms, Are Great!)}

In [5]:
class KVPair:
    def __init__(self, k, v):
        self.key = k
        self.val = v

    def __eq__(self, other):
        return self.key == other.key

    def __str__(self):
        return "("+str(self.key)+", "+str(self.val)+")" 

class HashMap:
    def __init__(self, d=10):    # a default array length of 10
        self.inArray = [LinkedList() for i in range(d)] 
        self.size = 0
        self.threshold = 0.75    # a default threshold of 0.75

    def hash(self, d):
        return hash(d) % len(self.inArray)
        
    def __str__(self):
        s = "{"
        for i in range(len(self.inArray)):
            x = self.inArray[i].toSeqString()
            if x != "": s += x+","
        if s == "{": return "{}"
        return s[:len(s)-1]+"}"
        
    # The search function is almost identical to the one for HashTable.
    # We first find the slot where the pair with key k could be by hashing k.
    # We then search in the corresponding linked list for a pair with key k, 
    # by simply searching in it for the pair KVPair(k,None): this will work
    # because equality of KVPairs is determined by comparing their keys.
    def search(self, k):
        i = self.hash(k)
        j = self.inArray[i].search(KVPair(k,None)) 
        if j == -1: return False
        return True

    def put(self, k, v):
        l = self.inArray[self.hash(k)]
        ptr = l.head
        while ptr:
            if ptr.data.key == k:
                ptr.data.val = v
                return
            ptr = ptr.next
        l.append(KVPair(k, v))
        self.size += 1
        
    def get(self, k):
        l = self.inArray[self.hash(k)]
        ptr = l.head
        while ptr:
            if ptr.data.key == k:
                return ptr.data
            ptr = ptr.next

    def remove(self, k):
        l = self.inArray[self.hash(k)]
        if l.head:
            if l.head.data.key == k:
                p = l.head.data
                if l.head.next:
                    l.head = l.head.next
                else:
                    l.head = None
                self.size -= 1
                return p

            ptr = l.head
            while ptr.next:
                if ptr.next.data.key == k:
                    p = ptr.next.data
                    if ptr.next.next:
                        ptr.next = ptr.next.next
                    else:
                        ptr.next = None
                    self.size -= 1
                    return p
                ptr = ptr.next

    def _resizeUp(self):
        oldArray = self.inArray
        self.inArray = [LinkedList() for i in range(2*len(oldArray))]
        for i in range(len(oldArray)):
            while oldArray[i].length > 0:
                d = oldArray[i].remove(0)
                self.inArray[self.hash(d)].insert(0,d)
        
        
h = HashMap()
h.put(10,12); print(h, h.size, h.search(10), h.search(42))
h.put(10,13); h.put(42,12); print(h, h.size, h.search(10), h.search(42))
h.put("Algorithms","Are Great!"); print(h)
print('Get:', h.get(10))
print('Remove:', h.remove(10), h)
print('Remove:', h.remove('Algorithms'), h)

{(10, 12)} 1 True False
{(10, 13),(42, 12)} 2 True True
{(10, 13),(42, 12),(Algorithms, Are Great!)}
Get: (10, 13)
Remove: (10, 13) {(42, 12),(Algorithms, Are Great!)}
Remove: (Algorithms, Are Great!) {(42, 12)}
